<a href="https://colab.research.google.com/github/nyanta012/demo/blob/main/Structured_outputs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install openai

In [2]:
import getpass
import os
from typing import List, Optional

from pydantic import BaseModel
from openai import OpenAI
from pydantic import BaseModel, Field


if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

OpenAI API Key:··········


# 基本的な使い方

In [8]:
class ResponseStep(BaseModel):
    steps: List[str]
    answer: int

In [9]:
client = OpenAI()

response = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    temperature=0,
    messages=[
        {
            "role": "system",
            "content": """あなたは優秀なAIです""",
        },
        {
            "role": "user",
            "content": "10個のりんごを二人で分けると1人は何個になる？",
        },
    ],
    response_format=ResponseStep,
)

In [10]:
response.choices[0].message.parsed.steps

['まず、10個のりんごを2人で分けるという問題を考えます。',
 'りんごの総数は10個です。',
 '2人で分けるので、10個のりんごを2で割ります。',
 '計算式は 10 ÷ 2 です。',
 'この計算を行うと、答えは5になります。',
 'したがって、1人あたりのりんごの数は5個です。']

In [14]:
response.choices[0].message.parsed.answer

5

# スコア算出

In [17]:
class Score(BaseModel):
    positive: int
    negative: int
    confidence: float = Field(...,description="回答の自信度を0から1の間で記入する")


class UserEmotion(BaseModel):
    """Userの立場で考える"""

    understanding_point: str = Field(..., description="理解できた点")
    non_understanding_point: str = Field(..., description="理解できなかった点")


class ResponseStep(BaseModel):
    score: List[Score]
    user_feedback: List[UserEmotion]
    explanation: str = Field(..., description="回答のみを出力する")

In [18]:
response = client.beta.chat.completions.parse(
        model="gpt-4o-2024-08-06",
        temperature=0,
        messages=[
            {
                "role": "system",
                "content": """あなたは優秀なAIです""",
            },
            {
                "role": "user",
                "content": "10個のりんごを100人で分けると1人は何個になる？",
            },
        ],
        response_format=ResponseStep,
    )

In [19]:
response.choices[0].message.parsed.score

[Score(positive=1, negative=0, confidence=0.9)]

In [20]:
response.choices[0].message.parsed.user_feedback[0].understanding_point

'りんごを100人で分けると、1人あたりのりんごの数が計算できた。'

In [21]:
response.choices[0].message.parsed.user_feedback[0].non_understanding_point

'りんごの数が10個しかないのに、100人で分けることができるのか疑問に思った。'

In [22]:
response.choices[0].message.parsed.explanation

'10個のりんごを100人で分けると、1人あたりのりんごの数は0.1個になります。これは、10を100で割った結果です。'

In [23]:
if response.choices[0].message.parsed.score[0].confidence > 0.5:
    print(response.choices[0].message.parsed.explanation)
else:
    print("no answer")

10個のりんごを100人で分けると、1人あたりのりんごの数は0.1個になります。これは、10を100で割った結果です。


# RAGとかAgentsでの利用

In [24]:
class ReasoningStep(BaseModel):
    reasoning: str
    action: str = Field(..., description="行動を単語で記入する")


class ActionStep(BaseModel):
    reasoning: str
    action: str = Field(..., description="行動を単語で記入する")
    query: Optional[str] = Field(
        description="RAGを検索する場合はその時のクエリを記入する"
    )


class ResponseStep(BaseModel):
    reasoning_steps: list[ReasoningStep]
    action_steps: list[ActionStep]

In [25]:
client = OpenAI()

completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {
            "role": "system",
            "content": """
            あなたは優秀なRAGの検索クエリを抽出する言語モデルです。ユーザーのクエリから目的を考えて、どのような行動をするべき考えてください。
            段階的な行動が必要である場合は、ステップ毎に何をするべきかを考えてください。
            また、その他知るべき内容があれば人間に質問を行うこともできます
            利用できるデータベースは下記です。
              - にゃんたの年齢、性別、趣味の情報が入ったデータベース
              - にゃんたの普段の行動の情報が入ったデータベース""",
        },
        {"role": "user", "content": "にゃんたはどんな人ですか？"},
    ],
    response_format=ResponseStep,
)

In [26]:
completion.choices[0].message.parsed.reasoning_steps

[ReasoningStep(reasoning='ユーザーはにゃんたについて知りたいようなので、まずデータベースで持っているにゃんたに関する情報を探し出す必要があります。', action='情報収集'),
 ReasoningStep(reasoning='データベースには2種類の情報がある。にゃんたの基本的なプロファイル情報（年齢、性別、趣味）と、普段の行動に関する情報です。まず、プロファイル情報を取得してユーザーに提供するのが適切です。', action='プロフィール検索')]

In [27]:
completion.choices[0].message.parsed.action_steps

[ActionStep(reasoning='まず、にゃんたの年齢、性別、趣味の情報を検索します。これが基礎的なパーソナル情報だからです。', action='検索', query='にゃんたの年齢, 性別, 趣味'),
 ActionStep(reasoning='プロフィール情報の次に、必要であれば普段の行動に関する情報を追加できます。これによって、にゃんたの生活スタイルや特性をより深く理解できます。', action='行動検索', query='にゃんたの普段の行動')]

In [28]:
for action in completion.choices[0].message.parsed.action_steps:
    print(f"検索クエリ：{action.query}")

検索クエリ：にゃんたの年齢, 性別, 趣味
検索クエリ：にゃんたの普段の行動
